# 0. Import libraries

In [ ]:
import os
import sys
import pypsa
import pandas as pd
import geopandas as gpd
import pycountry
import logging
import warnings
warnings.filterwarnings("ignore")

# 1. Define functions

In [ ]:
def create_folder(directory):
    """ Input:
            directory - path to directory
        Output:
            N/A
    """
    # Create the folder if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Folder '{directory}' created.")
    else:
        print(f"Folder '{directory}' already exists.")


def get_country_list(directory):
    """ Input:
            directory - path to results directory
        Output:
            List of countries in two-letter format
    """
    filenames = os.listdir(directory)
    country_names = set([x[:2] for x in filenames])
    return sorted(list(country_names))


def get_network_path(country_code, horizon, directory):
    """ Input:
            country_code - two-letter country code
            horizon - planning horizon
            directory - path to results directory
        Output:
            filepath - full path to network
    """
    foldername = f"{country_code}_{horizon}/networks"
    filenames = os.listdir(os.path.join(directory, foldername))
    assert len(filenames) == 1, print("Only 1 network per folder is allowed!")
    filepath = os.path.join(directory, foldername, filenames[0])
    return filepath


def get_base_network_path(country_code, horizon, directory):
    """ Input:
            country_code - two-letter country code
            horizon - planning horizon
            directory - path to results directory
        Output:
            filepath - full path to base network
    """
    foldername = f"{country_code}_{horizon}/"
    filenames = os.listdir(os.path.join(directory, foldername))

    filepath = os.path.join(directory, foldername, "base.nc")
    return filepath


def load_network(filepath):
    """ Input:
            filepath - full path to the network
        Output:
            n - PyPSA network
    """
    try:
        n = pypsa.Network(filepath)
        logging.info(f"Loading {filepath}")
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None
    return n


def get_total_demand(n):
    """ Input:
            n - network
        Output:
            demand - total electricity demand in TWh
    """
    demand = n.loads_t.p_set.multiply(n.snapshot_weightings.objective, axis=0).sum().sum() / 1e6
    return demand.round(4)


def get_optimal_capacities(n):
    """ Input:
            n - network
        Output:
            capacities - total electricity generation capacities in GW
    """
    # generator capacities
    gen_capacities = n.generators.groupby("carrier").p_nom_opt.sum()
    # storage unit capacities
    storage_capacities = n.storage_units.groupby("carrier").p_nom_opt.sum()
    # total capacities
    capacities = (pd.concat([gen_capacities, storage_capacities], axis=0) / 1e3).round(4)
    # drop load shedding
    if "load" in n.generators.carrier.unique():
        capacities.drop("load", inplace=True)
    return capacities


def get_generation_mix(n):
    """ Input:
            n - network
        Output:
            generation_mix - amount of consumed electricity generated by different carrier in TWh
    """
    # electricity generation by generators
    gen_generation = n.generators_t.p.multiply(n.snapshot_weightings.objective, axis=0).groupby(n.generators.carrier, axis=1).sum().sum()
    # electricity generation by storage unit
    storage_generation = n.storage_units_t.p.multiply(n.snapshot_weightings.objective, axis=0).groupby(n.storage_units.carrier, axis=1).sum().sum()
    # total generation mix
    generation_mix = pd.concat([gen_generation, storage_generation], axis=0) / 1e6
    return generation_mix.round(4)


def get_country_name(country_code):
    """ Input:
            country_code - two letter code of the country
        Output:
            country - corresponding name of the country
    """
    country = pycountry.countries.get(alpha_2=country_code)
    return country.name if country else None


def get_historic_demand(country_code, horizon):
    """ Input:
            country_code - two letter code of the country
        Output:
            demand - electricity demand in TWh from EIA (https://www.eia.gov/international/data/world/electricity/electricity-consumption)
    """
    # load EIA data
    eia_data = pd.read_csv(DATA_DIR+"EIA_demands.csv")
    eia_data.rename(columns={"Unnamed: 1":"country"}, inplace=True)
    eia_data["country"] = eia_data["country"].str.strip()
    # get the name of the country
    country_name = get_country_name(country_code)
    # select the demand for given country and year in TWh
    demand = eia_data.query("country == @country_name")[str(horizon)].values[0]
    return float(demand)


def get_historic_capacities(country_code, horizon):
    """ Input:
            country_code - two letter code of the country
        Output:
            capacity - generation capacities in MW from EIA (https://www.eia.gov/international/data/world/electricity/electricity-capacity)
    """
    # load EIA data
    eia_data = pd.read_csv(DATA_DIR+"EIA_installed_capacities.csv")
    eia_data.rename(columns={"Unnamed: 1":"country"}, inplace=True)
    eia_data["country"] = eia_data["country"].str.strip()
    # get the name of the country
    country_name = get_country_name(country_code)
    # find index where country starts
    country_index = eia_data.query("country == @country_name").index[0]
    capacities = eia_data.iloc[country_index+1:country_index+14][["country", str(horizon)]]
    return capacities


def get_historic_generation(country_code, horizon):
    """ Input:
            country_code - two letter code of the country
        Output:
            generation  - generation capacities in TWh from EIA (https://www.eia.gov/international/data/world/electricity/electricity-generation)
    """
    # load EIA data
    eia_data = pd.read_csv(DATA_DIR+"EIA_electricity_generation.csv")
    eia_data.rename(columns={"Unnamed: 1":"country"}, inplace=True)
    eia_data["country"] = eia_data["country"].str.strip()
    # get the name of the country
    country_name = get_country_name(country_code)
    # find index where country starts
    country_index = eia_data.query("country == @country_name").index[0]
    generation = eia_data.iloc[country_index+1:country_index+18][["country", str(horizon)]]
    return generation


def get_network_length(n_base):
    """ Input:
            n_base - base network of the country
        Output:
            length - total network length in km
            voltage_ratings - voltage ratings in kV
    """
    length = n_base.lines.length.sum().astype(int)
    voltage_ratings = n_base.lines.v_nom.astype(int).sort_values().unique()
    return length, voltage_ratings
    

def real_network_length(country_code):
    """ Input:
            country_code - two letter code of the country
        Output:
            length - total network length in km
            voltage_ratings - voltage ratings in kV
    """
    line_length = {"AU":56192, "BR":179297, "CN":1604838, "CO":29169, 
                   "DE":35796, "IN":706348, "IT":75246, "MX":109747, 
                   "NG":25633, "US":682812, "ZA": 0} # Eg: https://www.power-technology.com/data-insights/top-five-transmission-line-projects-in-colombia/
    voltage_ratings = {"AU": [66, 132, 275, 330], "BR": [138, 230, 345, 440, 500, 600, 750, 800],
                       "CN": [110, 220, 330, 500, 750, 800, 1000], "CO": [110, 115, 220, 230, 500],
                       "DE": [110, 220, 380], "IN": [66, 132, 220, 400, 765], 
                       "IT": [132, 150, 220, 380], "MX": [115, 230, 400], "NG": [132, 330],
                       "US": [69, 115, 138, 230, 345, 500, 765], "ZA": [66, 88, 132, 220, 330]}
    return line_length[country_code], voltage_ratings[country_code]

    
def compare_capacities(network_capacities, historic_capacities):
    """ Input:
            network_capacities - capacities obtained from the network in GW
            historic_capacities - capacities obtained from EIA in GW
        Output:
            capacities - DataFrame comparing two data
    """
    # bring historic data into format
    historic_capacities["country"] = historic_capacities.country.str.replace("(million kW)","").str.strip()
    historic_capacities.set_index("country", inplace=True)
    historic_capacities.columns = ["EIA Data"]
    historic_capacities.rename(index={"Capacity":"Total capacity", 
                                      "Hydroelectricity":"Hydro", 
                                      "Biomass and waste":"Biomass", 
                                      "Hydroelectric pumped storage":"PHS"}, inplace=True)
    historic_capacities.drop(index=["Renewables", "Non-hydroelectric renewables", 
                                    "Geothermal", "Solar, tide, wave, fuel cell", "Tide and wave"], inplace=True)
    historic_capacities = historic_capacities.loc[["Nuclear", "Fossil fuels", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Total capacity"], :]
    
    # bring capacities from the network into format
    all_carriers = ["nuclear", "coal", "lignite", "CCGT", "OCGT", "hydro", "ror", "PHS", "solar", "offwind-ac", "offwind-dc", "onwind", "biomass"]
    network_capacities = network_capacities.reindex(all_carriers, fill_value=0)
    network_capacities.rename(index={"nuclear":"Nuclear", "solar":"Solar", "biomass":"Biomass"}, inplace=True)
    network_capacities["Fossil fuels"] = network_capacities[["coal", "lignite", "CCGT", "OCGT"]].sum()
    network_capacities["Hydro"] = network_capacities[["hydro", "ror"]].sum()
    network_capacities["Wind"] = network_capacities[["offwind-ac", "offwind-dc", "onwind"]].sum()
    network_capacities = network_capacities.loc[["Nuclear", "Fossil fuels", "Hydro", "PHS", "Solar", "Wind", "Biomass"]]
    network_capacities["Total capacity"] = network_capacities.sum()
    network_capacities.name = "PyPSA Model"
    network_capacities = network_capacities.to_frame()

    # merge two data
    capacities = pd.concat([network_capacities, historic_capacities], axis=1).astype(float)
    capacities["Error (%)"] = (100*(capacities["PyPSA Model"] - capacities["EIA Data"])/capacities["EIA Data"]).astype(float).round(2)
    capacities.fillna(0, inplace=True)
    capacities.index.name = "Capacities [GW]"
    return capacities.round(2)


def compare_demands(network_demand, historic_demand):
    """ Input:
            network_demand - demand obtained from the network in TWh
            historic_demand - demand obtained from EIA in TWh
        Output:
            demand - DataFrame comparing two data
    """
    demand = pd.DataFrame(columns=["PyPSA Model", "EIA Data"])
    demand.loc["Electricity Demand [TWh]", ["PyPSA Model", "EIA Data"]] = [network_demand, historic_demand]
    demand["Error (%)"] = (100*(demand["PyPSA Model"] - demand["EIA Data"])/demand["EIA Data"]).astype(float)
    return demand.round(2)


def compare_generation(network_generation, historic_generation):
    """ Input:
            network_generation - electricity generation obtained from the network in TWh
            historic_generation - electricity generation obtained from EIA in TWh
        Output:
            generation - DataFrame comparing two data
    """
    # bring historic data into format
    historic_generation["country"] = historic_generation.country.str.replace("(billion kWh)","").str.strip()
    historic_generation.set_index("country", inplace=True)
    historic_generation.columns = ["EIA Data"]
    historic_generation = historic_generation.astype(float)
    historic_generation.rename(index={"Generation":"Total generation", 
                                      "Hydroelectricity":"Hydro", 
                                      "Biomass and waste":"Biomass", 
                                      "Hydroelectric pumped storage":"PHS"}, inplace=True)
    historic_generation.drop(index=["Fossil fuels", "Renewables", "Non-hydroelectric renewables", 
                                    "Geothermal", "Solar, tide, wave, fuel cell", "Tide and wave"], inplace=True)
    historic_generation.loc["Load shedding"] = None
    historic_generation.loc["Natural gas"] = historic_generation.loc[["Natural gas", "Other gases"]].sum()
    historic_generation = historic_generation.loc[["Nuclear", "Coal", "Natural gas", "Oil", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Load shedding", "Total generation"], :]
    
    # bring generation from the network into format
    all_carriers = ["nuclear", "coal", "lignite", "CCGT", "OCGT", "hydro", "ror", "PHS", "solar", "offwind-ac", "offwind-dc", "onwind", "biomass", "load"]
    network_generation = network_generation.reindex(all_carriers, fill_value=0)
    network_generation.rename(index={"nuclear":"Nuclear", "solar":"Solar", "biomass":"Biomass", "load":"Load shedding"}, inplace=True)
    network_generation["Coal"] = network_generation[["coal", "lignite"]].sum()
    network_generation["Natural gas"] = network_generation[["CCGT", "OCGT"]].sum()
    network_generation["Hydro"] = network_generation[["hydro", "ror"]].sum()
    network_generation["Wind"] = network_generation[["offwind-ac", "offwind-dc", "onwind"]].sum()
    network_generation = network_generation.loc[["Nuclear", "Coal", "Natural gas", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Load shedding"]]
    network_generation["Total generation"] = network_generation.sum()
    network_generation.name = "PyPSA Model"
    network_generation = network_generation.to_frame()

    # merge two data
    generation = pd.concat([network_generation, historic_generation], axis=1).astype(float)
    generation["Error (%)"] = (100*(generation["PyPSA Model"] - generation["EIA Data"])/generation["EIA Data"]).astype(float).round(2)
    generation.fillna(0, inplace=True)
    generation = generation.loc[["Nuclear", "Coal", "Natural gas", "Oil", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Load shedding", "Total generation"], :].round(2)
    return generation


def compare_network_lines(network_length, network_voltages, real_length, real_voltages):
    """ Input:
            network_length - network length in km
            network_voltages - network voltages in kV
            real_length - real network length in km
            real_voltages - real network voltages in kV
        Output:
            df_network - DataFrame comparing two data
    """
    voltage_ratings = sorted(list(set(network_voltages) | set(real_voltages)))
    df_network = pd.DataFrame(index=["Line length [km]"]+voltage_ratings)
    df_network.loc["Line length [km]", ["PyPSA Model", "Cited Sources"]] = [network_length, real_length]
    df_network.loc["Line length [km]", "Error (%)"] = (100*(network_length - real_length)/real_length).astype(float).round(2)
    df_network.loc[network_voltages, "PyPSA Model"] = "+"
    df_network.loc[real_voltages, "Cited Sources"] = "+"
    return df_network

In [ ]:
# directory to data and result folders
DATA_DIR = os.path.join(os.getcwd(), "../data/")
RESULTS_DIR = os.path.join(os.getcwd(), "../pypsa_data/results")
BASE_DIR = os.path.join(os.getcwd(), "../pypsa_data/networks")
OUTPUT_DIR = os.path.join(os.getcwd(), "../results/")

# create results folder
create_folder(OUTPUT_DIR)

# get list of countries
country_code_list = get_country_list(RESULTS_DIR)

# list of horizons
horizons = [2021]


with pd.ExcelWriter(OUTPUT_DIR+'JI-GIS_country_validation.xlsx', engine='xlsxwriter') as writer:
    for country_code in country_code_list:
        for horizon in horizons:
            # get filepath of the network
            filepath = get_network_path(country_code, horizon, RESULTS_DIR)
            # get filepath to base network
            base_filepath = get_base_network_path(country_code, horizon, BASE_DIR)
            
            # load the network
            n = load_network(filepath)
            # load base network
            n_base = load_network(base_filepath)
    
            # get total electricity demand
            demand = get_total_demand(n)
    
            # get optimal generation capacities
            generation_capacities = get_optimal_capacities(n)
    
            # get generation mix
            generation_mix = get_generation_mix(n)

            # get network length and voltage ratings
            network_length, network_voltages = get_network_length(n_base)
    
            # get historic electricity consumption of the country
            historic_demand = get_historic_demand(country_code, horizon)
    
            # get historic generation capacities of the country
            historic_capacities = get_historic_capacities(country_code, horizon)
    
            # get historic generation mix of the country
            historic_generation = get_historic_generation(country_code, horizon)

            # get real network length and voltages
            real_length, real_voltages = real_network_length(country_code)
    
            # compare the data
            demand_comparison = compare_demands(demand, historic_demand)
            capacity_comparison = compare_capacities(generation_capacities, historic_capacities)
            generation_comparison = compare_generation(generation_mix, historic_generation)
            network_comparison = compare_network_lines(network_length, network_voltages, real_length, real_voltages)

        demand_comparison.to_excel(writer, sheet_name=country_code, startrow=0, startcol=0)
        capacity_comparison.to_excel(writer, sheet_name=country_code, startrow=3, startcol=0)
        generation_comparison.to_excel(writer, sheet_name=country_code, startrow=0, startcol=5)
        network_comparison.to_excel(writer, sheet_name=country_code, startrow=0, startcol=10)
